In [1]:
import numpy as np
import pandas as pd
import sys
sys.path.append("../fraud_detection/src/")
from util import s_to_time_format, string_to_datetime,hour_to_range
from tqdm import tqdm

def value_to_count(df_train, df_test, df_train_normal_cano_id, df_):
    # separate continuous feature and categorial features
    feats = ['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
       'flbmk', 'flg_3dsmk', 'hcefg', 'insfg', 'iterm', 'locdt',
       'mcc', 'mchno', 'ovrlt', 'scity', 'stocn', 'stscd', 'loctm_hour_of_day',
       'loctm_minute_of_hour', 'loctm_second_of_min'] 
    cont_feats = ['iterm', 
                  'locdt',
                  'loctm_hour_of_day',
                  'loctm_minute_of_hour', 
                  'loctm_second_of_min']
    feats = [f for f in feats if f not in cont_feats]
    # we only coner categorial features
    
    df = pd.concat([df_train[feats], df_test[feats]], axis = 0)
    for f in tqdm(feats):
        count_dict = df[f].value_counts(dropna = False).to_dict() 
        df_train_normal_cano_id[f] = df_train_normal_cano_id[f].apply(lambda v: count_dict[v])
        df_train[f] = df_train[f].apply(lambda v: count_dict[v])
        df_test[f] = df_test[f].apply(lambda v: count_dict[v])
        df_[f] = df_[f].apply(lambda v: count_dict[v])
    return df_train,df_test,df_train_normal_cano_id, df_

def feature_normalization_auto(df_train, df_test, df_train_normal_cano_id,df_):
    """
    return two inputs of autoencoder, one is for train and another one is for test
    """
    #from sklearn.preprocessing import MinMaxScaler, MaxAbsScaler
    feats = ['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
       'flbmk', 'flg_3dsmk', 'hcefg', 'insfg', 'iterm', 'locdt',
       'mcc', 'mchno', 'ovrlt', 'scity', 'stocn', 'stscd', 'loctm_hour_of_day',
       'loctm_minute_of_hour', 'loctm_second_of_min']
    df = pd.concat([df_train[feats], df_test[feats]], axis = 0)


    for f in tqdm(feats):
        try:
            #scaler = MinMaxScaler()
            max_ = df[f].max()
            min_ = df[f].min()
            df_train_normal_cano_id[f] = df_train_normal_cano_id[f].apply(lambda x: (x-min_)/(max_-min_))
            df_[f] = df_[f].apply(lambda x: (x-min_)/(max_-min_))
            #df_test[f] = df_test[f].apply(lambda x: (x-min_)/(max_-min_))
        except:
            print(f)
    return df_train_normal_cano_id,df_

def partition_(df, num_features):
    data = []
    for i in range(len(df)):
        out = None
        if i == 0:
            out = np.concatenate(((np.zeros((2,num_features))),df.iloc[:1].values))
        elif i== 1:
            out = np.concatenate(((np.zeros((1,num_features))),df.iloc[:i+1].values))
        else:
            out = df.iloc[i+1-3:i+1].values
        data.append(out)
    return data

def partition(df_, sequence_length = 3):
    feats = [f for f in df_.columns if f not in {"fraud_ind","cano_help","locdt_help"}]
    sequences = []
    for _, df in df_.groupby(by = "cano_help"):
        data = partition_(df[feats], num_features = len(feats))
        for d in data:
            sequences.append(d)
    return sequences

def get_sequence_dataframe(df):
    df_train_sequences = partition(df)
    df_train_sequences = np.concatenate(df_train_sequences)
    df_train_sequences = pd.DataFrame(df_train_sequences)
    return df_train_sequences


#-----------------------------
# load data
#-----------------------------
df_train = pd.read_csv("/data/yunrui_li/fraud/dataset/train.csv")
df_test = pd.read_csv("/data/yunrui_li/fraud/dataset/test.csv")


for df in [df_train, df_test]:
    # pre-processing
    df["loctm_"] = df.loctm.astype(int).astype(str)
    df.loctm_ = df.loctm_.apply(s_to_time_format).apply(string_to_datetime)
    # time-related feature
    df["loctm_hour_of_day"] = df.loctm_.apply(lambda x: x.hour)
    df["loctm_minute_of_hour"] = df.loctm_.apply(lambda x: x.minute)
    df["loctm_second_of_min"] = df.loctm_.apply(lambda x: x.second)

    # removed the columns no need
    df.drop(columns = ["loctm_", "loctm","txkey"], axis = 1, inplace = True)

df_train["cano_locdt_index"] = ["{}_{}_{}_{}_{}".format(str(i),str(j),str(k),str(l),str(m)) for i,j,k,l,m in zip(df_train.cano,
                                                                                   df_train.locdt,
                                                                                   df_train.loctm_hour_of_day,
                                                                                   df_train.loctm_minute_of_hour,
                                                                                   df_train.loctm_second_of_min,
                                                                                  )]
df_test["cano_locdt_index"] = ["{}_{}_{}_{}_{}".format(str(i),str(j),str(k),str(l),str(m)) for i,j,k,l,m in zip(df_test.cano,
                                                                                  df_test.locdt,
                                                                                  df_train.loctm_hour_of_day,
                                                                                  df_train.loctm_minute_of_hour,
                                                                                  df_train.loctm_second_of_min,
                                                                                 )]

df_train["cano_help"] = df_train.cano
df_test["cano_help"] = df_test.cano

df_train["locdt_help"] = df_train.locdt
df_test["locdt_help"] = df_test.locdt

df_train["loctm_hour_of_day_help"] = df_train.loctm_hour_of_day
df_test["loctm_hour_of_day_help"] = df_test.loctm_hour_of_day

df_train["loctm_minute_of_hour_help"] = df_train.loctm_minute_of_hour
df_test["loctm_minute_of_hour_help"] = df_test.loctm_minute_of_hour

df_train["loctm_second_of_min_help"] = df_train.loctm_second_of_min
df_test["loctm_second_of_min_help"] = df_test.loctm_second_of_min

#-----------------------------
# feature extraction
#-----------------------------
df = pd.concat([df_train, df_test], axis = 0)
df.sort_values(by = ["cano", "locdt","loctm_hour_of_day","loctm_minute_of_hour","loctm_second_of_min"], inplace = True)

#-----------------------------
# prepare training data
#-----------------------------
df_train.sort_values(by = ["cano", "locdt","loctm_hour_of_day","loctm_minute_of_hour","loctm_second_of_min"], inplace = True)

# df_train, df_test = value_to_count(df_train, df_test)
# df_train, df_test = feature_normalization_auto(df_train, df_test)

fraud_cano_id = df_train[df_train.fraud_ind == 1].cano.unique().tolist()

df_train_normal_cano_id = df_train[~df_train.cano.isin(fraud_cano_id)]
print ("number of training data",df_train_normal_cano_id.shape)

df_train, df_test, df_train_normal_cano_id, df = value_to_count(df_train, df_test,df_train_normal_cano_id, df)
df_train_normal_cano_id, df = feature_normalization_auto(df_train, df_test,df_train_normal_cano_id, df)

#-----------------------------
# post-processing
#-----------------------------
df.drop(columns = ["fraud_ind"], axis = 1, inplace = True)
df_train_normal_cano_id.drop(columns = ["fraud_ind"], axis = 1, inplace = True)
feats = ['acqic', 'bacno', 'cano', 'conam', 'contp', 'csmcu', 'ecfg', 'etymd',
   'flbmk', 'flg_3dsmk', 'hcefg', 'insfg', 'iterm', 'locdt',
   'mcc', 'mchno', 'ovrlt', 'scity', 'stocn', 'stscd', 'loctm_hour_of_day',
   'loctm_minute_of_hour', 'loctm_second_of_min'] + ["cano_locdt_index","cano_help","locdt_help",
#                                                      "loctm_hour_of_day_help",
#                                                      "loctm_minute_of_hour_help",
#                                                      "loctm_second_of_min_help",
                                                    ]

df = df[feats]
df_train_normal_cano_id = df_train_normal_cano_id[feats]

#-----------------------------
# get train/test data
#-----------------------------

X_train = get_sequence_dataframe(df_train_normal_cano_id)
Feature = get_sequence_dataframe(df)


/ldap_home/yunrui.li/.pyenv/versions/3.6.5/envs/deepts/lib/python3.6/site-packages/ipykernel_launcher.py:134: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.



number of training data (1390382, 30)


  0%|          | 0/18 [00:00<?, ?it/s]/ldap_home/yunrui.li/.pyenv/versions/3.6.5/envs/deepts/lib/python3.6/site-packages/ipykernel_launcher.py:25: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  0%|          | 0/23 [00:00<?, ?it/s]/ldap_home/yunrui.li/.pyenv/versions/3.6.5/envs/deepts/lib/python3.6/site-packages/ipykernel_launcher.py:48: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
100%|██████████| 23/23 [01:10<00:00,  3.08s/it]
/ldap_home/yunrui.li/.pyenv/versions/3.6.5/envs/deepts/lib/python3.6/site-packages/p

In [108]:
#-----------------------------
# modeling (unsupervised learning)
#-----------------------------
from __future__ import print_function
import numpy as np
import tensorflow as tf
import six
from timeit import default_timer as timer


class LSTM_Var_Autoencoder(object):

    def __init__(self, intermediate_dim=None, z_dim=None, n_dim=None,
                 stateful=False):
        """
        Args:
        intermediate_dim : LSTM cells dimension.
        z_dim : dimension of latent space.
        n_dim : dimension of input data.
        statefull : if true, keep cell state through batches.
        """
        
        if not intermediate_dim or not z_dim or not n_dim:
            raise ValueError("You should set intermediate_dim, z_dim"
                             "(latent space) dimension and your input"
                             "third dimension, n_dim."
                             " \n            ")

        tf.reset_default_graph()

        self.z_dim = z_dim
        self.n_dim = n_dim
        self.intermediate_dim = intermediate_dim
        self.stateful = stateful
        self.input = tf.placeholder(tf.float32, shape=[None, None, self.n_dim])
        self.batch_size = tf.placeholder(tf.int64)

        # tf.data api
        dataset = tf.data.Dataset.from_tensor_slices(self.input).repeat() \
            .batch(self.batch_size)
        self.batch_ = tf.placeholder(tf.int32, shape=[])
        self.ite = dataset.make_initializable_iterator()
        self.x = self.ite.get_next()
        self.repeat = tf.placeholder(tf.int32)

        def gauss_sampling(mean, sigma):
            with tf.name_scope("sample_gaussian"):
                eps = tf.random_normal(tf.shape(sigma), 0, 1, dtype=tf.float32)
            # It should be log(sigma / 2), but this empirically converges"
            # much better for an unknown reason"
                z = tf.add(mean, sigma * eps)
                return z

    # (with few modifications) from https://stackoverflow.com/questions

        def get_state_variables(batch_size, cell):
            # For each layer, get the initial state and make a variable out of it
            # to enable updating its value.
            state_variables = []
            for state_c, state_h in cell.zero_state(batch_size, tf.float32):
                state_variables.append(tf.nn.rnn_cell.LSTMStateTuple(
                    (state_c), (state_h)))
    # Return as a tuple, so that it can be fed to dynamic_rnn as an initial
    # state
            return tuple(state_variables)

        # Add an operation to update the train states with the last state
        # tensors
        def get_state_update_op(state_variables, new_states):
            update_ops = []
            for state_variable, new_state in zip(state_variables, new_states):
                update_ops.extend([state_variable[0] == new_state[0],
                                   state_variable[1] == new_state[1]])
            return tf.tuple(update_ops)

        # Return an operation to set each variable in a list of LSTMStateTuples
        # to zero
        def get_state_reset_op(state_variables, cell, batch_size):
            zero_states = cell.zero_state(batch_size, tf.float32)
            return get_state_update_op(state_variables, zero_states)

        weights = {
            'z_mean': tf.get_variable(
                "z_mean",
                shape=[
                    self.intermediate_dim,
                    self.z_dim],
                initializer=tf.contrib.layers.xavier_initializer()),
            'log_sigma': tf.get_variable(
                "log_sigma",
                shape=[
                    self.intermediate_dim,
                    self.z_dim],
                initializer=tf.contrib.layers.xavier_initializer())}
        biases = {
            'z_mean_b': tf.get_variable("b_mean", shape=[self.z_dim],
                                        initializer=tf.zeros_initializer()),
            'z_std_b': tf.get_variable("b_log_sigma", shape=[self.z_dim],
                                       initializer=tf.zeros_initializer())
        }

        with tf.variable_scope("encoder"):
            with tf.variable_scope("LSTM_encoder"):
                lstm_layer = tf.nn.rnn_cell.LSTMCell(
                    self.intermediate_dim,
                    forget_bias=1,
                    initializer=tf.contrib.layers.xavier_initializer(),
                    activation=tf.nn.relu)

        if self.stateful:
            self.batch_ = tf.placeholder(tf.int32, shape=[])
            # throws an error without MultiRNNCell
            layer = tf.nn.rnn_cell.MultiRNNCell([lstm_layer])
            states = get_state_variables(self.batch_, layer)
            outputs, new_states = tf.nn.dynamic_rnn(
                layer, self.x, initial_state=states, dtype=tf.float32)
            self.update_op = get_state_update_op(states, new_states)
            self.reset_state_op = get_state_reset_op(
                states, lstm_layer, self.batch_)
        else:
            outputs, _ = tf.nn.dynamic_rnn(lstm_layer, self.x, dtype="float32")

        # For each layer, get the initial state. states will be a tuple of
        # LSTMStateTuples.
        self.z_mean = tf.add(tf.matmul(
            outputs[:, -1, :], weights['z_mean']), biases['z_mean_b'])
        self.z_sigma = tf.nn.softplus(tf.add(tf.matmul(
            outputs[:, -1, :], weights['log_sigma']), biases['z_std_b']))
        self.z = gauss_sampling(self.z_mean, self.z_sigma)

        # from [batch_size,z_dim] to [batch_size, TIMESTEPS, z_dim]
        repeated_z = tf.keras.layers.RepeatVector(
            self.repeat, dtype="float32")(self.z)

        with tf.variable_scope("decoder"):
            if self.stateful:
                with tf.variable_scope('lstm_decoder_stateful'):
                    rnn_layers_ = [
                        tf.nn.rnn_cell.LSTMCell(
                            size,
                            initializer=tf.contrib.layers.xavier_initializer(),
                            forget_bias=1) for size in [
                            self.intermediate_dim,
                            n_dim]]
                    multi_rnn_cell_ = tf.nn.rnn_cell.MultiRNNCell(rnn_layers_)
                    states_ = get_state_variables(self.batch_, multi_rnn_cell_)
                self.x_reconstr_mean, new_states_ = tf.nn.dynamic_rnn(
                    cell=multi_rnn_cell_, inputs=repeated_z, initial_state=states_, dtype=tf.float32)
                self.update_op_ = get_state_update_op(states_, new_states_)
                self.reset_state_op_ = get_state_reset_op(
                    states_, multi_rnn_cell_, self.batch_)
            else:
                with tf.variable_scope('lstm_decoder_stateless'):
                    rnn_layers = [
                        tf.nn.rnn_cell.LSTMCell(
                            size,
                            initializer=tf.contrib.layers.xavier_initializer(),
                            forget_bias=1) for size in [
                            self.intermediate_dim,
                            n_dim]]
                    multi_rnn_cell = tf.nn.rnn_cell.MultiRNNCell(rnn_layers)
                self.x_reconstr_mean, _ = tf.nn.dynamic_rnn(
                    cell=multi_rnn_cell, inputs=repeated_z, dtype=tf.float32)

    def _create_loss_optimizer(self, opt, **param):
        with tf.name_scope("MSE"):
            reconstr_loss = tf.reduce_sum(
                tf.losses.mean_squared_error(
                    self.x, self.x_reconstr_mean))
        with tf.name_scope("KL_divergence"):
            latent_loss = - 0.5 * tf.reduce_sum(1 + self.z_sigma**2
                                               - self.z_mean**2
                                               + tf.log(1.e-8 + self.z_sigma**2), 1)
            self._cost = tf.reduce_mean(reconstr_loss + latent_loss)
        # apply gradient clipping
        tvars = tf.trainable_variables()
        grads, _ = tf.clip_by_global_norm(tf.gradients(self._cost, tvars), 10)
        self.train_op = opt(**param).apply_gradients(zip(grads, tvars))

    def fit(
            self,
            X,
            learning_rate=0.001,
            batch_size=100,
            num_epochs=200,
            opt=tf.train.AdamOptimizer,
            REG_LAMBDA=0,
            grad_clip_norm=10,
            optimizer_params=None,
            verbose=True):      
        
        if len(np.shape(X)) != 3:
            raise ValueError(
                'Input must be a 3-D array. I could reshape it for you, but I am too lazy.'
                ' \n            Use input.reshape(-1,timesteps,1).')
        if optimizer_params is None:
            optimizer_params = {}
            optimizer_params['learning_rate'] = learning_rate
        else:
            optimizer_params = dict(six.iteritems(optimizer_params))

        self._create_loss_optimizer(opt, **optimizer_params)
        lstm_var = tf.get_collection(
            tf.GraphKeys.TRAINABLE_VARIABLES,
            scope='LSTM_encoder')
        self._cost += REG_LAMBDA * tf.reduce_mean(tf.nn.l2_loss(lstm_var))

        config = tf.ConfigProto(allow_soft_placement=True)
        config.gpu_options.allow_growth = True

        self.sess = tf.Session(config=config)
        init = tf.global_variables_initializer()
        self.sess.run(init)
        print ("batch_size",batch_size)
        print ("self.input", self.input)
        self.sess.run(
            self.ite.initializer,
            feed_dict={
                self.input: X,
                self.batch_size: batch_size})
        batches_per_epoch = int(np.ceil(len(X) / batch_size))

        print("\n")
        print("Training...")
        print("\n")
        start = timer()

        for epoch in range(num_epochs):
            train_error = 0
            for step in range(batches_per_epoch):
                if self.stateful:
                    loss, _, s, _ = self.sess.run([self._cost, self.train_op, self.update_op, self.update_op_],
                                                  feed_dict={self.repeat: np.shape(X)[1], self.batch_: batch_size})
                else:
                    loss, _ = self.sess.run([self._cost, self.train_op], feed_dict={
                                            self.repeat: np.shape(X)[1]})
                #print ("loss", loss)
                train_error += loss
            if step == (batches_per_epoch - 1):
                mean_loss = train_error / batches_per_epoch

                if self.stateful:  # reset cell & hidden states between epochs
                    self.sess.run([self.reset_state_op],
                                  feed_dict={self.batch_: batch_size})
                    self.sess.run([self.reset_state_op_],
                                  feed_dict={self.batch_: batch_size})
            if epoch % 10 == 0 & verbose:
                print(
                    "Epoch {:^6} Loss {:0.5f}"  .format(
                        epoch + 1, mean_loss))
        end = timer()
        print("\n")
        print("Training time {:0.2f} minutes".format((end - start) / (60)))

    def reconstruct(self, X, get_error=False):
        self.sess.run(
            self.ite.initializer,
            feed_dict={
                self.input: X,
                self.batch_size: np.shape(X)[0]})
        if self.stateful:
            _, _ = self.sess.run([self.reset_state_op, self.reset_state_op_], feed_dict={
                                 self.batch_: np.shape(X)[0]})
            x_rec, _, _ = self.sess.run([self.x_reconstr_mean, self.update_op, self.update_op_], feed_dict={
                                        self.batch_: np.shape(X)[0], self.repeat: np.shape(X)[1]})
        else:
            x_rec = self.sess.run(self.x_reconstr_mean,
                                  feed_dict={self.repeat: np.shape(X)[1]})
        if get_error:
            squared_error = (x_rec - X)**2
            return x_rec, squared_error
        else:
            return x_rec

    def reduce(self, X):
        self.sess.run(
            self.ite.initializer,
            feed_dict={
                self.input: X,
                self.batch_size: np.shape(X)[0]})
        if self.stateful:
            _ = self.sess.run([self.reset_state_op], feed_dict={
                              self.batch_: np.shape(X)[0]})
            x, _ = self.sess.run([self.z, self.update_op], feed_dict={
                                 self.batch_: np.shape(X)[0], self.repeat: np.shape(X)[1]})
        else:
            x = self.sess.run(self.z)
        return x

In [109]:
X_train_ = X_train.iloc[:,:-1].values

In [110]:
X_train_ = X_train_.reshape(-1,3,23)

In [111]:
X_train_.shape

(1390382, 3, 23)

In [112]:
X_train_[3]

array([[0.24511543634190078, 0.007168458781362007, 0.007168458781362007,
        0.00286028602860286, 1.0, 1.0, 1.0, 0.7834072999927885, 1.0, 1.0,
        1.0, 1.0, 0.0, 0.025210084033613446, 1.0, 0.07716814714837252,
        1.0, 1.0, 1.0, 1.0, 0.6521739130434783, 0.7457627118644068,
        0.11864406779661017],
       [0.7945588015540944, 0.007168458781362007, 0.007168458781362007,
        0.047804239883447806, 1.0, 1.0, 1.0, 0.7834072999927885, 1.0,
        1.0, 1.0, 1.0, 0.0, 0.15966386554621848, 0.06928499002387728,
        0.0001098590665118177, 1.0, 0.11813955687056894, 1.0, 1.0,
        0.6086956521739131, 0.8983050847457628, 0.711864406779661],
       [0.24511543634190078, 0.007168458781362007, 0.007168458781362007,
        0.01238069752921238, 1.0, 1.0, 1.0, 0.7834072999927885, 1.0, 1.0,
        1.0, 1.0, 0.0, 0.23529411764705882, 1.0, 0.07716814714837252,
        1.0, 1.0, 1.0, 1.0, 0.6521739130434783, 0.3728813559322034,
        0.7288135593220338]], dtype=object)

In [113]:
vae = LSTM_Var_Autoencoder(intermediate_dim = 15,z_dim = 10, n_dim=23, stateful = False) #default stateful = False


In [ ]:
vae.fit(X_train_, learning_rate=0.001, batch_size = 100, num_epochs = 100, opt = tf.train.AdamOptimizer, REG_LAMBDA = 0,
            grad_clip_norm=10, optimizer_params=None, verbose = True)

batch_size 100
self.input Tensor("Placeholder:0", shape=(?, ?, 23), dtype=float32)


Training...


Epoch   1    Loss -493545247081190592.00000
Epoch   11   Loss nan


In [ ]:
Feature_ = Feature.iloc[:3,:-1].values

Feature_ = Feature_.reshape(-1,3,23)

In [ ]:
Feature_.shape

In [ ]:
x_reconstructed, recons_error = vae.reconstruct(Feature_, get_error = True) #returns squared error

x_reduced = vae.reduce(Feature_) #latent space representation


In [ ]:
x_reconstructed

In [ ]:
recons_error

In [ ]:
x_reduced.shape

In [ ]:
x_reduced

In [ ]:
18381/5/60